In [7]:
import os
import re
from datetime import datetime

def read_barcodes(file_shtr):
    with open(out_dir+file_shtr) as f:
        lines = f.readlines()
        barcodes = {}
        for line in lines:
            pos = line.strip().split(',')
            pos = [x.replace('"','') for x in pos]
            barcodes[pos[1]] = pos[0]
    return barcodes

def pack_order(file, meo_code, barcodes):
    with open(out_dir+file) as f:
        lines = f.readlines()
        order = {}
        order['id'] = ''
        order['items'] = {}
        for index, line in enumerate(lines):
            pos = line.strip().split('","')
            pos = [x.replace('"','') for x in pos]
            if order['id'] == '':
                order['id'] = pos[0]
            order['items'][str(index)] = {}
            order['items'][str(index)]['item_id'] = pos[1]
            order['items'][str(index)]['item_art'] = pos[2]
            order['items'][str(index)]['item_text'] = pos[3]
            order['items'][str(index)]['item_qnt'] = pos[4]
    
    out_file_name = in_dir+'Уп_зак_GOOD_[УПАКОВКА_ЗАКАЗОВ]'+order['id']+'.DAT'
    with open(out_file_name, 'w') as f:
        for index, pos in enumerate(order['items']):
            for one_item in range(int(order['items'][str(index)]['item_qnt'])):
                current_time = datetime.now().strftime("%H:%M:%S")
                write_string = order['id']+','+\
                               meo_code+','+\
                               barcodes[order['items'][str(index)]['item_id']]+','+\
                               order['items'][str(index)]['item_id']+','+\
                               '1'+','+\
                               current_time+'\n'
                f.write(write_string)
    return

def pack(files_pack, meo_code, barcodes):
    for file in files_pack:        
        pack_order(file, meo_code, barcodes)
        meo_code = 'MEO'+str(int(meo_code.split('MEO')[1])+1)
    new_free_meo_code = meo_code
    return new_free_meo_code

def del_pack_files(files_pack, file_shtr):
    for file in files_pack:
        os.remove(out_dir+file)
    os.remove(out_dir+file_shtr)

def read_meo(meo_file, plant):
    with open('meo_number.txt') as f:
        meo_code = f.read().replace('XXXX', plant)
    return meo_code

def write_meo(meo_file, new_free_meo_code):
    with open(meo_file, 'w') as f:
        new_free_meo_code = 'MEOXXXX'+new_free_meo_code[7:]
        f.write(new_free_meo_code)
    print('Следующий свободный код MEO (записан в файл): ',new_free_meo_code)
    return new_free_meo_code

def tsd(plant, meo_file, file_shtr):
    #Получаем список файлов для ТСД по маске
    files_pack = [f for f in os.listdir(out_dir) if re.match(r'\[УПАКОВКА_ЗАКАЗОВ\].*\.DAT', f)]
    
    #Начальный код MEO (после использования в файл запишется следующий свободный номер)
    meo_code = read_meo(meo_file, plant)
    
    #Читаем файл со штрих-кодами
    barcodes = read_barcodes(file_shtr)
    
    #Пакуем заказы в MEO и записываем в файл следующий незанятый код MEO
    new_free_meo_code = pack(files_pack, meo_code, barcodes)
    write_meo(meo_file, new_free_meo_code)
    
    #Удаляем исходные файлы упаковкиа
    del_pack_files(files_pack, file_shtr)

In [10]:
#Исходные данные по файлам комплектации
out_dir    = 'C:\OMNIMAIL\OUT\\'
in_dir     = 'C:\OMNIMAIL\IN\\'
file_shtr  = 'УПАКОВКА_ЗАКАЗОВ-SHTR.DAT'
meo_file   = 'meo_number.txt' #храним тут текущий свободный номер МЕО
plant      = '2059'

#Формируем файлы загрузки в ТСД
tsd(plant, meo_file, file_shtr)

Следующий свободный код MEO (записан в файл):  MEOXXXX01000060
